In [ ]:
"""
Module: MODIS LST Data Processing Script

Script Functionality:
This script processes MODIS Land Surface Temperature (LST) data from the MYD11A1 collection using Google Earth Engine (GEE). 
It filters the LST data for a specific date range and region of interest (ROI), applies a quality check on the data, and 
calculates both the median and 90th percentile values for the region. The results are visualized using a color palette and 
exported to Google Drive.

Input Parameters:
Region of Interest (ROI) in Shapefile format (uploaded as an Earth Engine asset).
MODIS MYD11A1 ImageCollection (MODIS/006/MYD11A1).

Dependencies:
Earth Engine API : pip install earthengine-api (for accessing MODIS LST data and functions)
Geemap : pip install geemap (for visualizing and interacting with GEE in Python)


Version Information:
Version: v1.0
Change Log: October 08 2024.
"""
import ee
import geemap
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

# Define the region of interest (roi)
roi = ee.FeatureCollection("projects/ee-meiyang150269/assets/boundary")  # Replace with your Shapefile asset path

# Set start and end dates
start_date = '2022-01-01'
end_date = '2023-01-01'

# Load the MODIS MYD11A1 image collection and select relevant bands
MYD11A1 = ee.ImageCollection('MODIS/006/MYD11A1') \
    .filter(ee.Filter.date(start_date, end_date)) \
    .select(['LST_Day_1km', 'QC_Day']) \
    .map(lambda img: img.clip(roi))

# Define the quality check function
def CheckQA(image):
    qa = image.select('QC_Day')
    goodmask1 = 1 << 0
    mask1 = qa.bitwiseAnd(goodmask1).eq(0)
    goodmask2 = 1 << 1
    mask2 = qa.bitwiseAnd(goodmask2).eq(0)
    return image.updateMask(mask1.And(mask2))

# Apply quality check
LST_afterQAcheck = MYD11A1.map(CheckQA).select('LST_Day_1km')

# Count the number of valid pixels
count = LST_afterQAcheck.reduce(ee.Reducer.count()).rename('count')

# Add the count as a band
LST_afterQAcheck = LST_afterQAcheck.map(lambda img: img.addBands(count))

# Function to check the count of valid observations
def CheckCount(image):
    count = image.select('count')
    mask = count.gte(190)
    return image.updateMask(mask)

# Apply count check
LST_CheckQAandCount = LST_afterQAcheck.map(CheckCount).select('LST_Day_1km')

# Compute the median and max of LST after QA and count checks
LST_CheckQAandCount_Median = LST_CheckQAandCount.reduce(ee.Reducer.max())
LST_Median = MYD11A1.select('LST_Day_1km').reduce(ee.Reducer.max())

# Unmask the output
LST_Median_Output = LST_CheckQAandCount_Median.unmask(LST_Median)

# Set palette
palette = ['#a50026', '#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837']

# Add the layer to the map
Map = geemap.Map()
Map.addLayer(LST_Median_Output, {'min': 13000, 'max': 16500, 'palette': palette}, 'LST Median Output')

# Compute the 90th percentile
LST_CheckQAandCount_90Per = LST_CheckQAandCount.reduce(ee.Reducer.percentile([90]))
LST_90Per = MYD11A1.select('LST_Day_1km').reduce(ee.Reducer.percentile([90]))

# Unmask the 90th percentile output
LST_90Per_Output = LST_CheckQAandCount_90Per.unmask(LST_90Per)

# Add 90th percentile output layer to the map
Map.addLayer(LST_90Per_Output, {'min': 13000, 'max': 16500, 'palette': palette}, 'LST 90Per Output')

# Export the median output to Google Drive
task = ee.batch.Export.image.toDrive(
    image=LST_Median_Output,
    description='LSTmax2022',  # LSTmedian2020 // min max median
    folder="LST",
    scale=1000,
    region=roi.geometry(),
    maxPixels=1e13,
    crs="EPSG:4326"
)
task.start()

# End of the script

resample

In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 08, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: LST Raster Resampling Script

Script Functionality:
This script resamples Land Surface Temperature (LST) raster data based on a reference raster file. 
It reads raster files from a specified input folder, resamples them to match the resolution and projection 
of the reference raster, and saves the resampled output files to a designated folder.

Input Parameters:
A reference raster file (.tif) with the desired target resolution and projection.
A folder containing LST raster files (.tif) that need to be resampled.

Dependencies:
GDAL :pip install gdal (for raster processing and reprojection)
Rasterio :pip install rasterio (optional, for additional raster operations)
NumPy :pip install numpy (for numerical array operations if needed)


Version Information:
Version: v1.0
Change Log: October 08 2024.
"""
from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\LST\0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\LST\0.005\Resample'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, '0.005_' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 03, 2024
This code is protected by copyright and may not be copied or distributed without permission.

Module: LST Raster Resampling Script

Script Functionality:
This script resamples Land Surface Temperature (LST) raster data files (.tif) located in a specified input folder. 
It resamples each raster to a lower resolution (0.05) by averaging values based on a defined scale factor. 
The processed raster files are saved in a designated output folder with updated filenames reflecting the new resolution.

Input Parameters:
input_folder: A folder containing LST raster files (.tif) to be resampled.
output_folder: A folder where the resampled raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 03, 2024.
"""

import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\11.3\LST"  # Replace with your input folder path
output_folder = r"D:\ghana\11.3\LST1"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

处理文件: D:\ghana\11.3\LST\LSTmax_0.01_2021.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmax_0.05_2021.tif
处理文件: D:\ghana\11.3\LST\LSTmax_0.01_2022.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmax_0.05_2022.tif
处理文件: D:\ghana\11.3\LST\LSTmedian_0.01_2021.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmedian_0.05_2021.tif
处理文件: D:\ghana\11.3\LST\LSTmedian_0.01_2022.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmedian_0.05_2022.tif
处理文件: D:\ghana\11.3\LST\LSTmin_0.01_2021.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmin_0.05_2021.tif
处理文件: D:\ghana\11.3\LST\LSTmin_0.01_2022.tif
文件已保存到: D:\ghana\11.3\LST1\LSTmin_0.05_2022.tif
所有文件处理完成。
